In [1]:
import sys
import os 
sys.path.append(os.path.dirname(sys.path[0]) + '/src')
import pandas as pd

train_df = pd.read_csv('../data/raw/sentipolc/training_set_sentipolc16.csv')
test_df = pd.read_csv('../data/raw/sentipolc/test_set_sentipolc16_gold2000.csv', escapechar='\\')

In [2]:
from models.bert_rep import BertRep
import numpy as np

bert_rep = BertRep()
train_df['hs'] = train_df.apply(lambda x: (bert_rep.hidden_states([x['text']])), axis=1)
test_df['hs'] = test_df.apply(lambda x: (bert_rep.hidden_states([x['text']])), axis=1)

: 

: 

In [ ]:
train_df.to_csv('../data/processed/sentipolc/subj_embeddings_train.csv', index = False, header=True)
test_df.to_csv('../data/processed/sentipolc/subj_embeddings_test.csv', index = False, header=True)

In [22]:
from models.bert_rep import BertRep
import numpy as np

bert_rep = BertRep()
train_df['hs'] = train_df.apply(lambda x: np.array(bert_rep.last_hidden_state([x['text']])).flatten(), axis=1)

test_df['hs'] = test_df.apply(lambda x: np.array(bert_rep.last_hidden_state([x['text']])).flatten(), axis=1)

In [17]:
X_train = np.array(train_df['hs'].iloc[-1]).flatten()
y_train = train_df['iro'].to_list()[:20]

X_test = np.array(test_df['hs'].iloc[-1]).flatten()
y_test = test_df['iro'].to_list()[:20]


In [10]:
from sklearn import svm
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from matplotlib import pyplot as plt

clf = svm.SVC(kernel='poly' ,C = 1, class_weight='balanced')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

pr_iro = precision_score(y_test, y_pred, pos_label=1)
pr_no_iro = precision_score(y_test, y_pred, pos_label=0)

rec_iro = recall_score(y_test, y_pred, pos_label=1)
rec_no_iro = recall_score(y_test, y_pred, pos_label=0)

f1_iro = f1_score(y_test, y_pred, pos_label=1)
f1_no_iro = f1_score(y_test, y_pred, pos_label=0)

acc = accuracy_score(y_test, y_pred,)

print('Precision iro: {0:.3f}'.format(pr_iro))
print('Precision no iro: {0:.3f}'.format(pr_no_iro))
print('Recall iro: {0:.3f}'.format(rec_iro))
print('Recall no_iro: {0:.3f}'.format(rec_no_iro))
print('F1 iro: {0:.3f}'.format(f1_iro))
print('F1 no_iro: {0:.3f}'.format(f1_no_iro))
print('F1 mean: {0:.3f}'.format((f1_iro+f1_no_iro)/2))
print('Accuracy: {0:.3f}'.format(acc))

cm = confusion_matrix(y_test, y_pred, labels=clf.classes_, normalize='true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                   display_labels=clf.classes_)
disp.plot()
plt.show()

ValueError: Expected 2D array, got 1D array instead:
array=[ 1.20803428  0.06221029  0.35572958 ... -0.34548464 -0.34337562
 -0.67750883].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.